In [5]:
import datetime
import geopandas as gpd
import neatnet

REGION_ID = 0
TARGET_EPSG = 25832

overture_streets_dir = "D:/Work/Github_Morphotopes/data/overture_streets/"
buildings_dir        = "D:/Work/Github_Morphotopes/data/"
out_streets_dir      = "D:/Work/Github_Morphotopes/data/streets/"

APPROVED_ROADS = [
    "living_street","motorway","motorway_link","pedestrian","primary","primary_link",
    "residential","secondary","secondary_link","tertiary","tertiary_link","trunk",
    "trunk_link","unclassified"
]

def to_drop_tunnel(row):
    tunnel_length_total = -1.0
    seg_len = row.geometry.length
    for flag in row.road_flags:
        if "values" in flag and "is_tunnel" in flag["values"]:
            tunnel_length_total = 0.0 if tunnel_length_total < 0 else tunnel_length_total
            if "between" in flag and flag["between"] is not None:
                s, e = flag["between"]
                tunnel_length_total += (e - s)
    if tunnel_length_total == 0.0:
        return True
    if tunnel_length_total > 0:
        return (tunnel_length_total * seg_len) > 50.0
    return False

print("Reading streets…")
streets = gpd.read_parquet(f"{overture_streets_dir}streets_{REGION_ID}.parquet")
if streets.crs is None:  # only if you know the source is WGS84
    streets = streets.set_crs(4326)
streets = streets.to_crs(TARGET_EPSG)

if "class" in streets.columns:
    streets = streets[streets["class"].isin(APPROVED_ROADS)].copy()

# Optional: drop tunnel segments if road_flags are present
if "road_flags" in streets.columns:
    tf = streets.loc[~streets["road_flags"].isna()].copy()
    if not tf.empty:
        mask = tf.apply(to_drop_tunnel, axis=1)
        streets = streets.drop(tf.index[mask])

# Keep minimal columns (if present)
keep = [c for c in ["id", "geometry", "class"] if c in streets.columns]
streets = streets.reset_index(drop=True)[keep]

print("Reading buildings…")
buildings = gpd.read_parquet(f"{buildings_dir}buildings_{REGION_ID}.parquet", columns=["geometry"])
if buildings.crs != streets.crs:
    buildings = buildings.to_crs(TARGET_EPSG)

print("Simplifying with neatnet…", datetime.datetime.now())
simplified = neatnet.neatify(
    streets,
    exclusion_mask=buildings.geometry,
    artifact_threshold_fallback=7,
)
out_path = f"{out_streets_dir}streets_{REGION_ID}.parquet"
simplified.to_parquet(out_path)
print("Wrote:", out_path, "| features:", len(simplified))


Reading streets…
Reading buildings…
Simplifying with neatnet… 2025-08-14 15:26:14.204569


c:\Users\Itrat\Miniconda3_new\Lib\site-packages\neatnet\simplify.py:1030: UserWarning: An error occured at location POINT (393218.63776685443 5708237.660304248). The artifact has not been simplified. The original message:
can only convert an array of size 1 to a Python scalar
  streets = neatify_singletons(


Wrote: D:/Work/Github_Morphotopes/data/streets/streets_0.parquet | features: 242


In [6]:
read_streets = gpd.read_parquet(out_streets_dir + 'streets_0.parquet' )

In [7]:
read_streets

,geometry,_status,id,class
0,"LINESTRING (393635.185 5708434.207, 393644.758...",changed,0891f12cd097ffff046fbf56d6e14ea6,pedestrian
1,"LINESTRING (393362.325 5708232.494, 393310.742...",changed,0891f12cd093ffff046fbe663da0c643,residential
2,"LINESTRING (392964.835 5709246.16, 392976.533 ...",changed,0871f12cd4ffffff046ffbf7a09cc4f5,primary
3,"LINESTRING (393227.881 5709274.944, 393230.324...",changed,0871f12cd4ffffff046fbc3ed3c902dc,primary
4,"LINESTRING (392903.213 5708354.156, 392916.302...",changed,0871f12cd4ffffff046ffbf92ed1914d,primary
...,...,...,...,...
237,"LINESTRING (393661.534 5708424.94, 393677.211 ...",changed,0891f12cd097ffff046fbf56d6e14ea6,pedestrian
238,"LINESTRING (393627.306 5708421.574, 393633.565...",changed,08c1f12cd09445ff047fff34d4a47d50,pedestrian
239,"LINESTRING (393661.534 5708424.94, 393663.582 ...",changed,08c1f12cd09445ff047fff34d4a47d50,pedestrian
240,"LINESTRING (393627.306 5708421.574, 393635.185...",new,None,None


In [8]:
output = read_streets.to_file(out_streets_dir + 'streets_0.gpkg', driver='GPKG')